This notebook allow you to print results of models in tables

In [1]:
# from twinews import config as twinewsConf
# twinewsConf.mongoLocation = "octods"

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [5]:
def basicPrint\
(
    models,
    *args, w=None, b=None,
    splitVersion=2,
    doNormalization=True,
    discardAt10=True, addNdcg=True,
    **kwargs,
):
    if w is not None and isinstance(w, str):
        w = [w]
    if b is not None and isinstance(b, str):
        b = [b]
    if isinstance(w, set):
        w = list(w)
    if isinstance(b, set):
        b = list(b)
    if len(args) > 0:
        if w is not None:
            w = list(w) + list(args)
        else:
            w = list(args)
    if discardAt10:
        if b is None:
            b = []
        b = b + ['ndcg@10$', 'p@10$']
    if addNdcg:
        if w is None:
            w = []
        w = w + ['ndcg']
    printReportKwargs = \
    {
        'whiteModels': models,
        'splitVersion': splitVersion,
        'noSubsampling': True,
        'whiteMetrics': w,
        'blackMetrics': b,
        'sortBy': 'ndcg',
        'doNormalization': doNormalization,
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

In [6]:
def printAll(*args, **kwargs):
    kwargs['onlyFields'] = {'id', 'model'}
    kwargs['blackModels'] = {'combin'}
    return basicPrint(None, *args, **kwargs)

# Misc

In [ ]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

In [ ]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

In [ ]:
# Removing combin models with maxUsers is not None:
if False:
    twinewsRankings = getTwinewsRankings()
    ids = twinewsRankings.keys()
    bp(ids)
    for currentId in ids:
        if 'combin' in currentId:
            meta = twinewsRankings.getMeta(currentId)
            if meta['maxUsers'] is not None:
                del twinewsRankings[currentId]

# Model lists per split version

In [7]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [8]:
printModelList(2)

BM25_yf Convnet DSSM_title DSSM_yf bert bm25 combin dbert-base dbert-ft doc2vec ideal infersent jaccard lda nmf random sent2vec stylo tfidf usent word2vec worst (22)


In [ ]:
printModelList(1)

# Reference models

In [ ]:
args = ('div', 'nov', 'ser')
basicPrint('ideal', *args, splitVersion=2)
basicPrint('random', *args, splitVersion=2)
basicPrint('worst', *args, splitVersion=2)

In [ ]:
basicPrint(['ideal', 'random', 'random'], *args, splitVersion=2)


# Ad-hoc models

In [ ]:
basicPrint('jaccard', 'div', b=['topic', 'jacc', 'style'])

In [ ]:
basicPrint('bm25', 'map', 'mrr', 'ndcg', 'p@100')

In [ ]:
basicPrint('BM25_yf')

In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [ ]:
basicPrint('lda')

In [ ]:
basicPrint('nmf')

In [ ]:
basicPrint('tfidf', 'map', 'mrr', 'p@100')

In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# All models for split version 2

In [ ]:
printAll('div', splitVersion=2)

# Combinations of models

In [ ]:
# Expected 156 rows:
basicPrint('combin')

# Deep Learning models

In [ ]:
basicPrint('DSSM_yf', noSubsampling=False)

In [ ]:
basicPrint('DSSM_title', noSubsampling=False)

# Reports on the test set (split version 1)

In [ ]:
args = ('div', 'nov', 'ser')
basicPrint('ideal', *args, splitVersion=1)
basicPrint('random', *args, splitVersion=1)
basicPrint('worst', *args, splitVersion=1)

In [ ]:
printAll(splitVersion=1)

In [ ]:
basicPrint('combin', 'nov', splitVersion=1)

# Trash

In [ ]:
def basicPrint_deprecated(model, *args, b=None, w=None, splitVersion=2, **kwargs):
    if w is not None and isinstance(w, str):
        w = [w]
    if b is not None and isinstance(b, str):
        b = [b]
    if w is not None and len(args) > 0:
        args = list(args) + list(w)
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': splitVersion,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        # 'allowedMetrics': {'ndcg', 'div@100', 'topic-div@100', 'swjacc-div@100', 'jacc-div@100'}, # None, {'ndcg', 'ndcg@10'}
        # 'allowedMetrics': {'ndcg', 'ndcg@100', 'p@100', 'mrr', 'map'},
        'discardedMetrics': {'map', 'mrr', 'ndcg@10', 'ndcg@100', 'p@10', 'p@100'},
        'discardedMetrics': {'ndcg@10', 'p@10'},
        'whiteMetricPatterns': None if len(args) == 0 else set(list(args) + ['ndcg']),
        'blackMetricPatterns': b,
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport_deprecated(**printReportKwargs)

In [ ]:
def printAll_deprecated(*args, **kwargs):
    kwargs['onlyFields'] = {'id', 'model'}
    kwargs['excludedModels'] = {'combin', 'ideal', 'worst', 'random'}
    return basicPrint_deprecated(None, *args, **kwargs)